In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to your dataset
dataset_train_path = '/Users/jonathanprasetyo/Documents/Snake Classifier/snake/train'
dataset_test_path = '/Users/jonathanprasetyo/Documents/Snake Classifier/snake/test'

# Define the input image dimensions
img_height, img_width = 224, 224
input_shape = (img_height, img_width, 3)

# Define the number of classes (snake species)
num_classes = len(os.listdir(dataset_train_path))

def create_snake_classifier_model():
    model = models.Sequential()
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


# Use ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    dataset_train_path,
    target_size=(img_height, img_width),
    batch_size=64,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    dataset_train_path,
    target_size=(img_height, img_width),
    batch_size=64,
    class_mode='categorical'
)


# Create and compile the model
model = create_snake_classifier_model()


# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=33,
    validation_data=test_generator
)

# Save the model
model.save('snake_classifier_model.h5')

Found 26120 images belonging to 135 classes.
Found 26120 images belonging to 135 classes.
Epoch 1/33
408/408 [==============================] - 1038s 3s/step - loss: 4.8391 - accuracy: 0.0172 - val_loss: 4.6589 - val_accuracy: 0.0261
Epoch 2/33
408/408 [==============================] - 1037s 3s/step - loss: 4.6041 - accuracy: 0.0339 - val_loss: 4.4917 - val_accuracy: 0.0490
Epoch 3/33
408/408 [==============================] - 1040s 3s/step - loss: 4.4762 - accuracy: 0.0515 - val_loss: 4.3564 - val_accuracy: 0.0678
Epoch 4/33
408/408 [==============================] - 1039s 3s/step - loss: 4.3223 - accuracy: 0.0707 - val_loss: 4.1378 - val_accuracy: 0.0991
Epoch 5/33
408/408 [==============================] - 1039s 3s/step - loss: 4.1356 - accuracy: 0.0962 - val_loss: 3.8261 - val_accuracy: 0.1462
Epoch 6/33
408/408 [==============================] - 1037s 3s/step - loss: 3.9277 - accuracy: 0.1329 - val_loss: 3.6374 - val_accuracy: 0.1849
Epoch 7/33
408/408 [==========================

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import zipfile
import os

# Replace 'your_zip_file.zip' with the actual name of your zip file
zip_file_path = '/Users/jonathanprasetyo/Documents/Snake Classifier/snake.zip'

# Replace 'your_extracted_folder' with the desired folder name
extracted_folder_path = '/Users/jonathanprasetyo/Documents/Snake Classifier/extract'

# Create the folder to extract files into
os.makedirs(extracted_folder_path, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

# List the contents of the extracted folder
extracted_contents = os.listdir(extracted_folder_path)
print(f"Contents of the extracted folder: {extracted_contents}")

In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
IMAGE_SIZE = [224, 224]

In [3]:
train_directory='/Users/jonathanprasetyo/Documents/Snake Classifier/snake/train'
validation_directory='/Users/jonathanprasetyo/Documents/Snake Classifier/snake/test'

In [ ]:
weights_path = '/kaggle/input/working/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'

# add preprocessing layer to the front of VGG
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights=weights_path, include_top=False)

# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False 

In [ ]:
#getting number of classes
folders = glob('/Users/jonathanprasetyo/Documents/Snake Classifier/snake/train/*')
len(folders)

In [ ]:
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

In [ ]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)

validation_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(train_directory,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

validation_set = validation_datagen.flow_from_directory(validation_directory,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

In [ ]:
print(len(training_set))
print(len(validation_set))

In [ ]:
r = model.fit(
  training_set,
  validation_data=validation_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(validation_set)
)

In [ ]:
model.save('SC.h5')